In [1]:
import shutil
import os, pickle
from config import config
from func import load_corpus
from rule_selection_func import rule_selection_func

unable to import 'smart_open.gcs', disabling that module


In [ ]:
validate = load_corpus.load_corpus('synthetic')

rule_corpus = 'Tablib_Sample_Large'
rule_type_list = ['embed', 'sbert', 'cta', 'doduo', 'pattern', 'pyfunc', 'validator']

# these are for accelerating the computation of sbert and doduo rules
# set them to where the intermediate results are stored (or where they will be stored if they are not available yet)
sbert_dist_val_embeddings_fname = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.sbert, f'{load_corpus.CORPUS_NAME}_dist_val_embeddings.pkl')
doduo_intermediate_result_dir = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.doduo)
doduo_dist_val_scores_fname = f'{load_corpus.CORPUS_NAME}_dist_val_scores.pickle'

rule_outlier_results = rule_selection_func.rule_utility_compute(validate, rule_corpus, rule_type_list, 
                                                                sbert_dist_val_embeddings_fname, doduo_intermediate_result_dir, doduo_dist_val_scores_fname)

In [3]:
results_file_path = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.validate_results, f'{load_corpus.CORPUS_NAME}_rule_{rule_corpus}_results.pickle')
with open(results_file_path, 'wb') as file:
    pickle.dump(rule_outlier_results, file)

In [4]:
results_file_path = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.validate_results, f'{load_corpus.CORPUS_NAME}_rule_{rule_corpus}_results.pickle')
with open(results_file_path, 'rb') as file:
    rule_outlier_results = pickle.load(file) 

In [5]:
rule_fname_list = [os.path.join(config.dir.project_base_dir, config.dir.project_base.sdc_output, f'{rule_corpus}_{rule_type}.pickle') for rule_type in rule_type_list]
rule_list = []
for rule_fname in rule_fname_list:
    with open(rule_fname, 'rb') as file:
        rule_list += pickle.load(file)

In [ ]:
# fine_selection

fine_select_params = {
    'cohenh_thres': 0.8,
    'conf_thres': 0.1,
    'num_rule_thres': 500,
    'significance_thres': 0.08/100000,
    'delta': 0.0001,
    'num_repeat_for_eval': 1000,
}

randomize_rounding_selection = rule_selection_func.fine_selection(rule_list, rule_outlier_results, fine_select_params)
selected_rules = randomize_rounding_selection()[0]

In [7]:
print(f'# selected SDC: {len(selected_rules)}')
print(selected_rules[:10])

# selected SDC: 504
[(('cta', 'state', 0.9, 0.15000000000000002), ('cta', 0), 2.3716958503205468, 0.02530004952529478, (0, 148, 256195, 42056)), (('cta', 'state', 0.9, 0.15000000000000002), ('cta', 0.002), 2.951366517646344, 0.02530004952529478, (0, 148, 295561, 2690)), (('cta', 'category', 0.8, 0.15000000000000002), ('cta', 0.002), 2.7036390799841046, 0.05780610513250667, (2, 120, 295836, 2441)), (('cta', 'country', 0.8, 0.7000000000000001), ('cta', 0), 1.9686467508191234, 0.060802523903021044, (22, 519, 256173, 41685)), (('cta', 'region', 0.8, 0.15000000000000002), ('cta', 0), 2.064106421985278, 0.06664488090027204, (3, 125, 256192, 42079)), (('cta', 'name', 0.8, 0.25), ('cta', 0), 1.963340494092737, 0.06723431717573151, (15, 347, 256180, 41857)), (('cta', 'region', 0.8, 0.1), ('cta', 0), 2.0970418611723227, 0.05367103787397927, (3, 157, 256192, 42047)), (('cta', 'county', 0.8, 0.1), ('cta', 0), 2.0379136553755246, 0.06877967448824501, (4, 141, 256192, 42062)), (('cta', 'position', 0